<a href="https://colab.research.google.com/github/charviadikar/AI_lab_1BM22CS012/blob/main/1BM22CS012_Week9_FOL_Resolution_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from typing import List, Tuple

def is_complementary(clause1: set, clause2: set) -> Tuple[bool, set]:
    """
    Check if two clauses are complementary and return the resolved clause if they are.
    """
    for literal in clause1:
        if f"~{literal}" in clause2:
            resolved = clause1.union(clause2)
            resolved.discard(literal)
            resolved.discard(f"~{literal}")
            return True, resolved
        elif literal.startswith("~") and literal[1:] in clause2:
            resolved = clause1.union(clause2)
            resolved.discard(literal)
            resolved.discard(literal[1:])
            return True, resolved
    return False, set()

def resolution(kb: List[set], goal: set) -> Tuple[bool, List[Tuple[set, List[int]]]]:
    """
    Perform resolution and build the proof tree.
    """
    proof_tree = []
    clauses = kb + [goal]  # Add negated goal to the KB
    new_clauses = []

    while True:
        # Generate new pairs of clauses
        pairs = [(clauses[i], clauses[j], (i, j)) for i in range(len(clauses)) for j in range(i + 1, len(clauses))]

        for clause1, clause2, (i, j) in pairs:
            complementary, resolved_clause = is_complementary(clause1, clause2)
            if complementary:
                if not resolved_clause:  # Empty clause found
                    proof_tree.append((resolved_clause, [i, j]))
                    return True, proof_tree
                if resolved_clause not in clauses and resolved_clause not in new_clauses:
                    new_clauses.append(resolved_clause)
                    proof_tree.append((resolved_clause, [i, j]))

        if not new_clauses:  # No progress, resolution failed
            return False, proof_tree

        clauses.extend(new_clauses)
        new_clauses = []

# Example case study
if __name__ == "__main__":
    # Knowledge Base (KB) in CNF format
    kb = [
        {"P", "Q"},       # Clause 1: P ∨ Q
        {"~P", "R"},      # Clause 2: ¬P ∨ R
        {"~R"},           # Clause 3: ¬R
    ]
    goal = {"~Q"}  # Negated goal: ¬Q

    success, proof_tree = resolution(kb, goal)
    if success:
        print("PROVED")
    else:
        print("CAN'T BE PROVED")

    print("\nProof Tree:")
    for i, (clause, parents) in enumerate(proof_tree):
        print(f"Step {i + 1}: {clause} from {parents}")

PROVED

Proof Tree:
Step 1: {'R', 'Q'} from [0, 1]
Step 2: {'P'} from [0, 3]
Step 3: {'~P'} from [1, 2]
Step 4: {'Q'} from [0, 6]
Step 5: {'R'} from [1, 5]
Step 6: set() from [5, 6]


In [12]:
from itertools import combinations

def unify(x, y, theta={}):
    """Unify two predicates using a substitution (theta)."""
    if theta is None:
        return None
    elif x == y:
        return theta
    elif isinstance(x, str) and x.islower():
        return unify_var(x, y, theta)
    elif isinstance(y, str) and y.islower():
        return unify_var(y, x, theta)
    elif isinstance(x, tuple) and isinstance(y, tuple) and len(x) == len(y):
        return unify(x[1:], y[1:], unify(x[0], y[0], theta))
    else:
        return None

def unify_var(var, x, theta):
    """Unify variable."""
    if var in theta:
        return unify(theta[var], x, theta)
    elif x in theta:
        return unify(var, theta[x], theta)
    else:
        theta[var] = x
        return theta

def resolve(clause1, clause2):
    """Resolve two clauses and return the resolvents."""
    resolvents = []
    for predicate1 in clause1:
        for predicate2 in clause2:
            theta = unify(predicate1, negate(predicate2))
            if theta is not None:
                new_clause = (substitute(clause1, theta) | substitute(clause2, theta)) - {predicate1, predicate2}
                resolvents.append(frozenset(new_clause))
    return resolvents

def negate(predicate):
    """Negate a predicate."""
    return ('not', predicate) if isinstance(predicate, str) else predicate[1]

def substitute(clause, theta):
    """Apply substitutions to a clause."""
    return {substitute_predicate(p, theta) for p in clause}

def substitute_predicate(predicate, theta):
    """Apply substitution to a single predicate."""
    if isinstance(predicate, str):
        return theta.get(predicate, predicate)
    else:
        return (predicate[0],) + tuple(theta.get(arg, arg) for arg in predicate[1:])

def resolution(kb, query):
    """Perform resolution to prove the query."""
    negated_query = frozenset({negate(query)})
    clauses = kb | {negated_query}
    new_clauses = set()

    while True:
        for clause1, clause2 in combinations(clauses, 2):
            resolvents = resolve(clause1, clause2)
            if frozenset() in resolvents:
                return True
            new_clauses.update(resolvents)
        if new_clauses.issubset(clauses):
            return False
        clauses |= new_clauses

kb = {
    frozenset({('Mother', 'Leela', 'Oshin')}),
    frozenset({('Alive', 'Leela')}),
    frozenset({('not','Mother', 'x','y')}),
    frozenset({('Parent','x','y')}),
    frozenset({('not','Parent', 'w', 'z')}),
    frozenset({('not','Alive','w','z')}),
    frozenset({('Older','w','z')}),
}

# Query: Prove John likes peanuts
query = ('Older', 'Leela', 'Older')

# Run resolution
result = resolution(kb, query)
if result:
    print("Proved by resolution: Hence, Leela is older than Oshin.")
else:
    print("Cannot prove. Leela is not older than Oshin.")




Proved by resolution: Hence, Leela is older than Oshin.
